# Análise das tendências políticas no município do Rio de Janeiro
## Por Omar Mesquita

Código do município do Rio de Janeiro: **3304557** (obtido no [site do IBGE](https://www.ibge.gov.br/cidades-e-estados/rj/rio-de-janeiro.html))

[Aqui](https://www.tse.jus.br/servicos-eleitorais/cartorios-e-zonas-eleitorais/pesquisa-a-zonas-eleitorais) pode-se ver as zonas eleitorais no **estado** do Rio de Janeiro. Por uma consulta
SQL podemos visualizar apenas zonas no **município** do Rio: <br><br>

```SQL

    SELECT ano, turno, sigla_uf, id_municipio, zona, numero_partido, resultado, votos
    FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio_zona`
    WHERE cargo='presidente'
      AND ano >= 2008
      AND sigla_uf='RJ'
      AND id_municipio = '3304557'
    LIMIT 100
```


In [1]:
# Por favor, execute essa célula antes de continuar
# para garantir as importações de módulos

import pandas as pd     # Leitura e tratamento de tabelas
import basedosdados as bd   # API para acessar dados do TSE
import os                   # Caminhos e I/O

# Determina o diretório no qual o caderno está sendo executado
currentDir = os.getcwd()

Devido à elevada quantidade de zonas eleitorais no município do Rio de Janeiro, faz-se necessário escolher aquelas que contenham a maior quantidade de pessoas votantes. Essa atitude viabiliza a análise dos dados e maximiza o espaço amostral do estudo *(e?)*

Podemos obter essa informação na página oficial do Tribunal Superior Eleitoral de [estatísticas do eleitorado](https://www.tse.jus.br/eleicoes/estatisticas/estatisticas-de-eleitorado/consulta-por-municipio-zona).

Para esse estudo, foi baixada uma tabela em formato `.csv` contendo dados da última eleição (**outubro de 2022**) no **estado do Rio de Janeiro (RJ)**

In [12]:

df = pd.read_csv(os.path.join(currentDir,"csv","quantidade_de_eleitores.csv"))
x = df.loc[df['Município'] == "RIO DE JANEIRO"]
print(x)

/home/omar/j/dataspell_env
            Mês/Ano  UF       Município  Zona  Quantidade  %  Unnamed: 6
98   Outubro - 2022  RJ  RIO DE JANEIRO     4      121744  0         949
99   Outubro - 2022  RJ  RIO DE JANEIRO     5      123307  0         961
100  Outubro - 2022  RJ  RIO DE JANEIRO     7      111138  0         866
101  Outubro - 2022  RJ  RIO DE JANEIRO     8       88360  0         689
102  Outubro - 2022  RJ  RIO DE JANEIRO     9      145837  1         137
103  Outubro - 2022  RJ  RIO DE JANEIRO    10       87452  0         682
104  Outubro - 2022  RJ  RIO DE JANEIRO    14       90888  0         709
105  Outubro - 2022  RJ  RIO DE JANEIRO    16      123286  0         961
106  Outubro - 2022  RJ  RIO DE JANEIRO    17      115326  0         899
107  Outubro - 2022  RJ  RIO DE JANEIRO    21       92673  0         722
108  Outubro - 2022  RJ  RIO DE JANEIRO    22      117508  0         916
109  Outubro - 2022  RJ  RIO DE JANEIRO    23       85107  0         663
110  Outubro - 2022  RJ 